# Baseline-inspired pulses (interactive)

Adjust the controls and click **Play** to rerun the Bloch-sphere animation with a detuning shift and scaled Rabi pulses. The pulse plot below is synchronized: a marker walks along each control as the state evolves.


In [ ]:

import json
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
import numpy as np
from IPython.display import HTML, display
from qutip import Bloch, Qobj, sesolve, sigmax, sigmay, sigmaz

rcParams["animation.embed_limit"] = 120  # MB embed budget
rcParams["animation.html"] = "jshtml"


def find_baseline_dir():
    for root in [Path.cwd(), *Path.cwd().parents]:
        candidate = root / "data" / "baselines" / "_baseline_crab"
        if candidate.exists():
            return candidate
    raise FileNotFoundError("Could not find data/baselines/_baseline_crab from this working directory")


BASELINE_DIR = find_baseline_dir()
metadata = json.loads((BASELINE_DIR / "metadata.json").read_text())
arrays = np.load(BASELINE_DIR / "arrays.npz")

# Static operators and initial state (density converted to ket for sesolve)
SX = sigmax()
SY = sigmay()
SZ = sigmaz()
HX = 0.5 * SX
HZ = 0.5 * SZ

rho0 = Qobj(arrays["rho0"])
_, eigvecs = rho0.eigenstates()
psi0 = eigvecs[-1]

delta_ref = float(np.max(np.abs(arrays["Delta0"])))

T_DEFAULT = 5.0
N_SAMPLES = 1000
MAX_FRAMES = 50
COLORS = ["#1f77b4", "#ff7f0e"]

print(
    f"Baseline loaded from {BASELINE_DIR} (Nt={metadata['summary']['Nt']}, T0={arrays['T'].item()} us)."
)


Baseline loaded from /home/yehon/projects/grape-crab-qoc/data/baselines/_baseline_crab (Nt=201, T0=0.1 us).


In [ ]:

# Simulation helpers


def time_grid(T_value):
    return np.linspace(0.0, T_value, N_SAMPLES)


def base_protocols(T_pi, T_rap):
    t_pi = time_grid(T_pi)
    t_rap = time_grid(T_rap)

    pulse_pi = np.sin(np.pi * t_pi / T_pi) ** 2
    detuning_pi = np.zeros_like(t_pi)
    omega_ref_pi = np.pi / np.trapezoid(pulse_pi, t_pi)  # area-normalized pi pulse
    pi_omega = omega_ref_pi * pulse_pi
    pi_delta = detuning_pi

    pulse_rap = np.sin(np.pi * t_rap / T_rap) ** 2
    detuning_rap = np.cos(np.pi * t_rap / T_rap)
    rap_omega = 50.0 *  pulse_rap
    rap_delta = 50.0 * detuning_rap

    return [
        {"label": "pi pulse: Omega sin^2, Delta=0", "omega": pi_omega, "delta": pi_delta, "t_full": t_pi},
        {"label": "RAP: Omega sin^2, Delta cos", "omega": rap_omega, "delta": rap_delta, "t_full": t_rap},
    ]


def simulate_protocol(omega_t, delta_t, t):
    H = [
        [HX, lambda tt, args=None: np.interp(tt, t, omega_t)],
        [HZ, lambda tt, args=None: np.interp(tt, t, delta_t)],
    ]
    result = sesolve(H, psi0, t, e_ops=[SX, SY, SZ])
    return np.vstack(result.expect)


def make_protocols(T_pi, T_rap, omega_scale, detuning_shift):
    proto_defs = base_protocols(T_pi, T_rap)
    protocols = []
    for base in proto_defs:
        t_full = base["t_full"]
        omega = omega_scale * base["omega"]
        delta = base["delta"] + detuning_shift
        coords_full = simulate_protocol(omega, delta, t_full)
        protocols.append(
            {
                "label": base["label"],
                "t_full": t_full,
                "omega_full": omega,
                "delta_full": delta,
                "coords_full": coords_full,
            }
        )
    return protocols


def pulse_limits(protocols):
    max_val = 0.0
    for proto in protocols:
        max_val = max(
            max_val,
            float(np.max(np.abs(proto["omega_full"]))),
            float(np.max(np.abs(proto["delta_full"]))),
        )
    span = max_val if max_val > 0 else 1.0
    span *= 1.1
    return -span, span


def build_animation(protocols, interval_ms=50):
    t_max = max(float(proto["t_full"][-1]) for proto in protocols)
    n_frames = max(100, MAX_FRAMES)
    global_t = np.linspace(0.0, t_max, n_frames)

    fig = plt.figure(figsize=(12, 7))
    gs = fig.add_gridspec(2, 2, height_ratios=[3, 1], hspace=0.32, wspace=0.3)
    bloch_axes = [fig.add_subplot(gs[0, i], projection="3d") for i in range(len(protocols))]
    pulse_axes = [fig.add_subplot(gs[1, i]) for i in range(len(protocols))]

    bloch_objs, bloch_lines, bloch_points, bloch_vectors = [], [], [], []
    for ax, proto, color in zip(bloch_axes, protocols, COLORS):
        bloch = Bloch(fig=fig, axes=ax)
        bloch.sphere_alpha = 0.14
        bloch.frame_color = "#888888"
        bloch.frame_alpha = 0.2
        bloch.vector_width = 5
        bloch.point_size = [30]
        bloch.point_marker = ["o"]
        bloch.vector_color = ["#d62728"]
        bloch.point_color = ["#d62728"]
        bloch.xlabel = ["x", ""]
        bloch.ylabel = ["y", ""]
        bloch.zlabel = ["z", ""]
        bloch.render()
        ax.set_title(proto["label"])
        trace, = ax.plot([], [], [], color=color, lw=2, alpha=0.9)
        point, = ax.plot([], [], [], marker="o", color="#d62728", markersize=6)
        vector = ax.quiver(0, 0, 0, 0, 0, 0, color="#d62728", lw=2.2, arrow_length_ratio=0.25)
        bloch_objs.append(bloch)
        bloch_lines.append(trace)
        bloch_points.append(point)
        bloch_vectors.append(vector)

    omega_lines, delta_lines, omega_markers, delta_markers = [], [], [], []
    ylim_low, ylim_high = pulse_limits(protocols)
    for proto, color, pax in zip(protocols, COLORS, pulse_axes):
        omega_line, = pax.plot(proto["t_full"], proto["omega_full"], color=color, label="Omega")
        delta_line, = pax.plot(
            proto["t_full"], proto["delta_full"], color=color, linestyle="--", alpha=0.8, label="Delta"
        )
        omega_marker, = pax.plot([], [], marker="o", color=color)
        delta_marker, = pax.plot([], [], marker="x", color=color, alpha=0.9)
        pax.set_title(f"{proto['label']} pulses")
        pax.set_xlabel("Time (us)")
        pax.set_ylabel("Amplitude (rad/us)")
        pax.set_xlim(0, float(proto["t_full"][-1]) if len(proto["t_full"]) else 1.0)
        pax.set_ylim(ylim_low, ylim_high)
        pax.legend(loc="upper right")
        omega_lines.append(omega_line)
        delta_lines.append(delta_line)
        omega_markers.append(omega_marker)
        delta_markers.append(delta_marker)

    def update(frame):
        t_global = global_t[frame % len(global_t)]
        artists = []
        for i, proto in enumerate(protocols):
            t_full = proto["t_full"]
            coords_full = proto["coords_full"]
            t_now = min(t_global, float(t_full[-1]))

            x = np.interp(t_now, t_full, coords_full[0])
            y = np.interp(t_now, t_full, coords_full[1])
            z = np.interp(t_now, t_full, coords_full[2])

            path_mask = t_full <= t_now
            bloch_lines[i].set_data(coords_full[0, path_mask], coords_full[1, path_mask])
            bloch_lines[i].set_3d_properties(coords_full[2, path_mask])
            bloch_points[i].set_data([x], [y])
            bloch_points[i].set_3d_properties([z])

            bloch_vectors[i].remove()
            bloch_vectors[i] = bloch_axes[i].quiver(
                0,
                0,
                0,
                x,
                y,
                z,
                color="#0004ff",
                lw=2.2,
                arrow_length_ratio=0.25,
                normalize=False,
            )
            artists.extend([bloch_lines[i], bloch_points[i], bloch_vectors[i]])

            omega_markers[i].set_data([t_now], [np.interp(t_now, t_full, proto["omega_full"])])
            delta_markers[i].set_data([t_now], [np.interp(t_now, t_full, proto["delta_full"])])
            artists.extend([omega_markers[i], delta_markers[i]])
        return artists

    anim = animation.FuncAnimation(fig, update, frames=len(global_t), interval=interval_ms, blit=False)
    return fig, anim, len(global_t)


In [ ]:

# Interactive controls: sliders update parameters silently; Play triggers the simulation.

omega_scale = widgets.FloatSlider(
    value=1.0, min=0.25, max=2.5, step=0.05, description="Omega scale", readout_format=".2f", continuous_update=False
)
detuning_shift = widgets.FloatSlider(
    value=0.0,
    min=-15.0,
    max=15.0,
    step=0.25,
    description="Delta shift",
    readout_format=".2f",
    continuous_update=False,
)
T_pi_slider = widgets.FloatSlider(
    value=T_DEFAULT,
    min=0.1,
    max=10.0,
    step=0.1,
    description="T pi (us)",
    readout_format=".2f",
    continuous_update=False,
)
T_rap_slider = widgets.FloatSlider(
    value=T_DEFAULT,
    min=0.1,
    max=10.0,
    step=0.1,
    description="T RAP (us)",
    readout_format=".2f",
    continuous_update=False,
)
play_button = widgets.Button(description="Play", icon="play", button_style="success")
status = widgets.HTML()


def update_status(*_):
    status.value = (
        f"<b>Parameters</b>: Omega scale={omega_scale.value:.2f}, "
        f"Delta shift={detuning_shift.value:.2f} rad/us, "
        f"T_pi={T_pi_slider.value:.2f} us, T_RAP={T_rap_slider.value:.2f} us. "
        "Click Play to run the simulation."
    )


for slider in (omega_scale, detuning_shift, T_pi_slider, T_rap_slider):
    slider.observe(update_status, names="value")
update_status()

out = widgets.Output()


def on_play(_):
    with out:
        out.clear_output(wait=True)
        status.value = (
            f"Running with Omega scale={omega_scale.value:.2f}, "
            f"Delta shift={detuning_shift.value:.2f} rad/us, "
            f"T_pi={T_pi_slider.value:.2f} us, T_RAP={T_rap_slider.value:.2f} us..."
        )
        try:
            protocols = make_protocols(T_pi_slider.value, T_rap_slider.value, omega_scale.value, detuning_shift.value)
            fig, anim, n_frames = build_animation(protocols)
            html = anim.to_jshtml()
            plt.close(fig)
            status.value = (
                f"Complete. Frames={n_frames}, "
                f"T_pi={T_pi_slider.value:.2f} us, T_RAP={T_rap_slider.value:.2f} us. "
                "Press Play again after adjusting sliders to rerun."
            )
            display(HTML(html))
        except Exception as exc:  # noqa: BLE001
            status.value = f"<span style='color:#cc0000;'>Simulation failed: {exc}</span>"
            raise


play_button.on_click(on_play)

layout = widgets.VBox(
    [
        widgets.HTML("<b>Adjust sliders, then press Play.</b>"),
        widgets.HBox([omega_scale, detuning_shift, T_pi_slider, T_rap_slider, play_button]),
        status,
        out,
    ]
)

display(layout)
